In [1]:
import gym
import torch, numpy as np
from torch import nn
import tianshou as ts
from agents import TwoAgentPolicy
from agents.lib_agents import SinePolicy
from utils import make_render_env, make_env, make_discrete_env, make_render_discrete_env

In [2]:
# create environment

env = make_env()
train_envs = ts.env.DummyVectorEnv([make_env for _ in range(1)])
test_envs = ts.env.DummyVectorEnv([make_render_env for _ in range(1)])

/home/devanshu/.local/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
# setup network

class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        # print("obs", obs)
        # print("view", obs.view(batch, -1))
        logits = self.model(obs.view(batch, -1))
        return logits, state

In [4]:
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n

In [5]:
net1 = Net(state_shape, action_shape)
net2 = Net(state_shape, action_shape)

In [6]:
optim1 = torch.optim.Adam(net1.parameters(), lr=1e-3)
optim2 = torch.optim.Adam(net2.parameters(), lr=1e-3)

In [7]:
# p1 = ts.policy.DQNPolicy(net1, optim1, discount_factor=0.9, estimation_step=3, target_update_freq=320)
# p1 = testPolicy()
p1 = SinePolicy()
p2 = ts.policy.DQNPolicy(net2, optim2, discount_factor=0.9, estimation_step=3, target_update_freq=320)
policy = TwoAgentPolicy(observation_space=env.observation_space, action_space=env.action_space, policies=(p1, p2))

In [8]:
# setup collector

train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(2000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

In [9]:
# training

result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64)
print(f'Finished training! Use {result}')

Epoch #1: 10001it [00:15, 627.54it/s, env_step=10000, len=90, loss=0.022, n/ep=1, n/st=10, rew=-1.00]                           


Epoch #1: test_reward: -1.000000 ± 0.000000, best_reward: 0.780000 ± 0.625780 in #0


Epoch #2: 10001it [00:16, 619.33it/s, env_step=20000, len=0, loss=0.002, n/ep=0, n/st=10, rew=0.00]                           


Epoch #2: test_reward: -1.000000 ± 0.000000, best_reward: 0.780000 ± 0.625780 in #0


Epoch #3: 10001it [00:16, 606.00it/s, env_step=30000, len=0, loss=0.000, n/ep=0, n/st=10, rew=0.00]                           


Epoch #3: test_reward: -1.000000 ± 0.000000, best_reward: 0.780000 ± 0.625780 in #0


Epoch #4:  34%|###4      | 3420/10000 [00:05<00:10, 606.10it/s, env_step=33420, len=0, loss=0.000, n/ep=0, n/st=10, rew=0.00]


KeyboardInterrupt: 